In [13]:
import pandas as pd

In [14]:
df_fdc = pd.read_excel('./data/raw/Sensor_DATA_DSS_반출.xlsx')
df_image = pd.read_excel('./data/raw/CO_IMAGE_DSS_반출.xlsx')

In [15]:
print(f'fdc = {df_fdc.shape}, image = {df_image.shape}')

fdc = (1309, 276), image = (435, 6)


In [16]:
import zipfile
import os

def extract_images_with_ids(excel_file, output_folder):
    df = pd.read_excel(excel_file, header=None)

    with zipfile.ZipFile(excel_file, 'r') as z:
        file_list = z.namelist()
        image_files = [f for f in file_list if f.startswith('xl/media/')]
        image_file_index = 0
        for i in range(1, len(df), 2):  # 홀수 행 (0-based index: row 1, row 3, ...)
            id_row = df.iloc[i]  # 아이디가 있는 행
            image_row = df.iloc[i - 1]  # 해당 아이디의 이미지가 있는 행

            for col_index in range(len(id_row)):
                if pd.isna(id_row[col_index]):  # 아이디가 없는 셀은 건너뜀
                    continue
                
                if image_file_index >= len(image_files):  # 이미지 파일이 더 이상 없으면 종료
                    print("No more images to process.")
                    return

                id_value = id_row[col_index]  # 아이디 값 추출
                image_filename = f"{id_value}.png"
                image_output_path = os.path.join(output_folder, image_filename)

                with open(image_output_path, 'wb') as f:
                    f.write(z.read(image_files[image_file_index]))

                image_file_index += 1  # 다음 이미지로 이동

# 이미지 추출할 엑셀 파일과 이미지 저장할 폴더 경로
excel_file = './data/raw/CO_IMAGE_DSS_반출.xlsx'
output_folder = './data/image'

# 이미지 추출 함수 실행
extract_images_with_ids(excel_file, output_folder)